# 安装依赖的第三方库

In [ ]:
!pip install -r requirements.txt

# Data process
从原始70个特征中选择出39个相关特征，并保存为半精度的pt文件

In [3]:
# 将路径加入系统变量
import sys
import os
path = os.path.abspath('.').split('\\')[0:-1]
path = '\\'.join(path)
sys.path.append(path)
sys.path.append(os.path.abspath('.'))
print(os.path.abspath('.'))
# print(sys.path)

F:\Desktop\weather_code\code


In [ ]:
import os
import torch
import numpy as np
import xarray as xr

channels = xr.open_zarr('../data/weather_round1_train_2007').x.channel.values

# 通过均值和方差，找出的与目标特征相关的特征通道
target_features = {'t2m': ['z150', 'z200', 'z250', 'z300', 'z400', 'z500', 't100', 't150', 't250',
                           't300', 't400', 't500', 't600', 't700', 'u50', 'r150', 't2m'],   # >0.9
                   'u10': ['t100', 't150', 't700', 'u50', 'u850', 'u925', 'u1000', 'r50', 'r100', 't2m', 'u10'],    # >0.75
                   'v10': ['v1000', 'v10', 'v925', 'u1000', 'u10', 'u925'],   # 均值相关的只有三个，加上了方差相关的
                   'msl': ['z1000', 'u50', 'u100', 'u150', 'u200', 'u250', 'u300', 'u400', 'r50',
                           'r100', 'r150', 'r200', 'msl', 'tp'],    # >0.8
                   'tp': ['t100', 't150', 't250', 't300', 't400', 't500', 'u50', 'r50', 'r100',
                          'r150', 'r200', 'r850', 'r925', 'r1000', 'msl', 'tp'] # >0.8
                   }

# 把字典的所有值合并成一个没有重复元素的集合
all_features = list(set(sum(target_features.values(), [])))
all_features = [feature for feature in channels if feature in all_features]
print(len(all_features), '\n', all_features)


def load_dataset(s_year, e_year):
    ds = []
    for y in range(s_year, e_year+1):
        data_name = os.path.join(f'../data/weather_round1_train_{y}')
        x = xr.open_zarr(data_name, consolidated=True)
        print(f'{data_name}, {x.time.values[0]} ~ {x.time.values[-1]}')
        ds.append(x)
    ds = xr.concat(ds, 'time')
    return ds

for year in range(2007, 2012):
    data = load_dataset(year,year).x
    data = data.sel(channel=all_features).values.astype(np.float16)
    data = torch.from_numpy(data)
    torch.save(data, f'../data/{year}_39.pth')

# 训练部分（Training
修改SimVP模型，使其预测39*2个特征，输出5*2个目标变量。其中2是时间步。由于需要预测未来20步的数据，这里通过多次不同的预测实验，选择使用10个相同架构的模型，独立预测。

In [ ]:
import os
import time
from tqdm import tqdm
from pathlib import Path

import xarray as xr
import numpy as np

import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from timm.scheduler import create_scheduler
from utils.tools import getModelSize, load_model, save_model, EMA
from utils.eval import fourcastnet_pretrain_evaluate
# from tensorboardX import SummaryWriter

from model.SimVP.model import SimVP as Model

SAVE_PATH = Path('./checkpoint/')
SAVE_PATH.mkdir(parents=True, exist_ok=True)

t_start = time.time()


""" 准备训练集和验证集 """
channels = xr.open_zarr('../data/weather_round1_train_2007').x.channel.values
# 通过均值和方差，找出的与目标特征相关的特征通道
target_features = {'t2m': ['z150', 'z200', 'z250', 'z300', 'z400', 'z500', 't100', 't150', 't250',
                           't300', 't400', 't500', 't600', 't700', 'u50', 'r150', 't2m'],   # >0.9
                   'u10': ['t100', 't150', 't700', 'u50', 'u850', 'u925', 'u1000', 'r50', 'r100', 't2m', 'u10'],    # >0.75
                   'v10': ['v1000', 'v10', 'v925', 'u1000', 'u10', 'u925'],   # 均值相关的只有三个，加上了方差相关的
                   'msl': ['z1000', 'u50', 'u100', 'u150', 'u200', 'u250', 'u300', 'u400', 'r50',
                           'r100', 'r150', 'r200', 'msl', 'tp'],    # >0.8
                   'tp': ['t100', 't150', 't250', 't300', 't400', 't500', 'u50', 'r50', 'r100',
                          'r150', 'r200', 'r850', 'r925', 'r1000', 'msl', 'tp'] # >0.8
                   }

# 把字典的所有值合并成一个没有重复元素的集合
all_features = list(set(sum(target_features.values(), [])))
all_features = [feature for feature in channels if feature in all_features]
print(len(all_features), '\n', all_features)


class ERA5(Dataset):
    def __init__(self, data, pred_s, pred_e):
        self.data = data
        self.pred_s = pred_s
        self.pred_e = pred_e

    def __len__(self):
        return self.data.shape[0] - 2 - 20 + 1

    def __getitem__(self, idx):
        x = self.data[idx:idx+2] # [B, t, c, h, w] == [B, 2, 70, h, w]
        y = self.data[idx+2+self.pred_s:idx+2+self.pred_e, -5:]   # [B,C,H,W] == [20, 5, h, w]

        return x, y


train_data = torch.load('../data/2007_39.pt')
print("Load training data...\n", train_data.shape)
for year in range(2008, 2011):
    data = torch.load(f'../data/{year}_39.pt')
    train_data = torch.cat((train_data, data), dim=0)
    print(train_data.shape)

val_data = torch.load(f'../data/2011_39.pt')
print('\nLoad validation data... \n', val_data.shape)


In [ ]:
def pretrain_one_epoch(epoch, start_step, model, criterion, data_loader, optimizer, loss_scaler, lr_scheduler, min_loss, writer=None, ema=None):
    loss_val = torch.tensor(0., device="cuda")
    count = torch.tensor(1e-5, device="cuda")

    model.train()
    global_step = epoch*len(data_loader)
    threshold = 0.001

    for step, batch in enumerate(tqdm(data_loader, leave=True, position=0)):
        if step < start_step:
            continue

        x, y = [x.cuda(non_blocking=True) for x in batch]

        with torch.cuda.amp.autocast():
            out = model(x)
            loss = criterion(out, y)
            if torch.isnan(loss).int().sum() == 0:
                count += 1
                loss_val += loss

        loss_scaler.scale(loss).backward()

        # # 记录每一层的梯度信息
        # for name, param in model.named_parameters():
        #     if param.grad is not None:
        #         nan_count = torch.isnan(param.grad).sum().item()
        #         below_threshold_count = (param.grad.abs() < threshold).sum().item()
        #         # writer.add_scalar(f'Gradient/{name}/NaN Count', nan_count, global_step+step)
        #         writer.add_scalar(f'Gradient/{name}/Below Threshold Count', below_threshold_count, global_step+step)


        loss_scaler.step(optimizer)
        loss_scaler.update()
        optimizer.zero_grad()
        ema.update()

    return loss_val.item() / count.item()


def train(args, pred_s, pred_e):

    data_train, data_val = ERA5(train_data, pred_s, pred_e), ERA5(val_data, pred_s, pred_e)

    model = Model().cuda()
    ema = EMA(model, 0.999)
    ema.register()
    param_sum, buffer_sum, all_size = getModelSize(model)
    print(f"Number of Parameters: {param_sum}, Number of Buffers: {buffer_sum}, Size of Model: {all_size:.4f} MB")
    optimizer = torch.optim.AdamW(model.parameters(), args.lr, weight_decay=args.weight_decay, betas=(0.9, 0.95))
    loss_scaler = torch.cuda.amp.GradScaler(enabled=True)
    lr_scheduler, _ = create_scheduler(args, optimizer)
    criterion = torch.nn.MSELoss()

    # writer = SummaryWriter(f'runs/model')
    # input = torch.randn((1, 2, 39, 161, 161)).cuda()
    # writer.add_graph(model, (input,))
    # writer.close()

    # load data
    train_dataloader = DataLoader(data_train, args.batch_size, num_workers=0, shuffle=True, pin_memory=True, drop_last=False)  # , num_workers=8, pin_memory=True
    val_dataloader = DataLoader(data_val, args.batch_size, num_workers=0, pin_memory=True, drop_last=False)

    # load
    start_epoch, start_step, min_loss, early_stop = load_model(model, ema, optimizer, lr_scheduler, loss_scaler, SAVE_PATH / f'backbone_best_{pred_s}_{pred_e}.pt')  # backbone_best
    print(f"Start pretrain for {args.pretrain_epochs} epochs, now {start_epoch}/{args.pretrain_epochs}")
    print(f'min_loss:{min_loss}')

    writer = None
    last_loss = min_loss
    early_stop = 0

    for epoch in range(start_epoch, args.pretrain_epochs):
        # 创建TensorBoard记录器
        # writer = SummaryWriter(f'runs/epoch{epoch}')

        t0 = time.time()
        train_loss = pretrain_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler, lr_scheduler, min_loss, writer, ema)  #
        t1 = time.time()
        # writer.close()
        start_step = 0
        lr_scheduler.step(epoch)

        val_loss = fourcastnet_pretrain_evaluate(val_dataloader, model, criterion, ema)

        print(f"Epoch {epoch} | Train loss: {train_loss:.6f}, Val loss: {val_loss:.6f}, Time: {t1-t0:.2f}s, Early stop: {early_stop}/3")
        if val_loss < min_loss:
            min_loss = val_loss
            save_model(model, epoch + 1, 0, optimizer, lr_scheduler, loss_scaler, min_loss, early_stop, ema, SAVE_PATH / f'backbone_best_{pred_s}_{pred_e}.pt')
        else:
            if last_loss < val_loss:
                early_stop += 1
            else:
                early_stop = 0
            last_loss = val_loss
            if early_stop >= 3:
                print(f'Early Stop at Epoch {epoch}')
                save_model(model, epoch + 1, 0, optimizer, lr_scheduler, loss_scaler, min_loss, early_stop, ema, SAVE_PATH / f'backbone_best_{pred_s}_{pred_e}.pt')
                break

In [ ]:
class AttrDict(dict):
    def __getattr__(self, name):
        if name in self:
            return self[name]
        raise AttributeError(f"'AttrDict' object has no attribute '{name}'")

    def __setattr__(self, name, value):
        self[name] = value

def get_args():
    args = dict(batch_size=44, pretrain_epochs=30, fintune_epochs=25, drop=0.0, drop_path=0.1,
                opt='adamw', opt_eps=1e-08, opt_betas=None, clip_grad=1, momentum=0.9, weight_decay=0.05, sched='cosine',
                lr=0.0005, lr_noise=None, lr_noise_pct=0.67, lr_noise_std=1.0, warmup_lr=1e-06, min_lr=1e-05,
                decay_epochs=30, warmup_epochs=5, cooldown_epochs=10, patience_epochs=10, decay_rate=0.1,
                color_jitter=0.4, aa='rand-m9-mstd0.5-inc1', smoothing=0.1, train_interpolation='bicubic',
                repeated_aug=False, reprob=0, remode='pixel', recount=1, resplit=False, fno_bias=False, fno_blocks=4,
                fno_softshrink=0.0, double_skip=False, tensorboard_dir=None, hidden_size=256, num_layers=12,
                checkpoint_activations=False, autoresume=False, num_attention_heads=1, ls_w=4, ls_dp_rank=16)
    args = AttrDict(args)
    return args


args = get_args()
for pred_s, pred_e in zip(range(0,22,2), range(2,22,2)):
    print(f'\npred_s: {pred_s}, pred_e: {pred_e}')
    t_start = time.time()
    train(args, pred_s, pred_e)
    t_end = time.time()
    print(f'Train model || pred_s: {pred_s}, pred_e: {pred_e}, total_time: {t_end - t_start}')

# 预测部分（Inference
使用保存的10个模型，分别预测对应的部分，然后拼接起来，得到最终的预测结果。

In [ ]:
target_features = {'t2m': ['z150', 'z200', 'z250', 'z300', 'z400', 'z500', 't100', 't150', 't250',
                           't300', 't400', 't500', 't600', 't700', 'u50', 'r150', 't2m'],   # >0.9
                   'u10': ['t100', 't150', 't700', 'u50', 'u850', 'u925', 'u1000', 'r50', 'r100', 't2m', 'u10'],    # >0.75
                   'v10': ['v1000', 'v10', 'v925', 'u1000', 'u10', 'u925'],   # 均值相关的只有三个，加上了方差相关的
                   'msl': ['z1000', 'u50', 'u100', 'u150', 'u200', 'u250', 'u300', 'u400', 'r50',
                           'r100', 'r150', 'r200', 'msl', 'tp'],    # >0.8
                   'tp': ['t100', 't150', 't250', 't300', 't400', 't500', 'u50', 'r50', 'r100',
                          'r150', 'r200', 'r850', 'r925', 'r1000', 'msl', 'tp'] # >0.8
                   }

# 把字典的所有值合并成一个没有重复元素的集合
all_features = list(set(sum(target_features.values(), [])))
all_features = [feature for feature in channels if feature in all_features]
print(len(all_features), '\n', all_features)
feature_idx = []
for feature in all_features:
    feature_idx.append(int(np.where(np.isin(channels, feature))[0][0]))
print(feature_idx)
# target_features = {key: [item for item in channels if item in value] for key, value in target_features.items()}

In [ ]:

# 把数据一次全部读取，放入dataset和dataloader
data = []
for idx in range(300):
    idx = str(idx).zfill(3)     # 将idx转换成3位数的字符串，在前面补0
    data.append(torch.load(f'../{idx}.pt', map_location='cpu'))


model_list = []
for s,e in zip(range(0,20,2),range(2,22,2)):
    model = Model((2, 39, 161, 161)).cuda()
    model.load_state_dict(torch.load(f'backbone_best_{s}_{e}.pt')['ema']['shadow'])
    model_list.append(model)


if not os.path.exists('../submit'):
    os.mkdir('../submit')

if not os.path.exists('../submit/output'):
    os.mkdir('../submit/output')


# inference
with torch.no_grad():
    for i in range(300):
        idx = str(i).zfill(3)
        data_in = data[i].float().unsqueeze(0).cuda()
        data_in = data_in[:, :, feature_idx]
        output = []
        for model in model_list:
            output.append(model(data_in))
        output = torch.cat(output, dim=1).squeeze(0).half()
        torch.save(output, f'../submit/output/{idx}.pt')
        print(f'save {idx}.pt ...')


## 将预测结果转成压缩文件

In [14]:
import zipfile, os
path = '../submit/output/'  # 要压缩的文件路径
zipName = '../submit/output.zip'  # 压缩后的zip文件路径及名称

# 创建一个新的zip文件
f = zipfile.ZipFile(zipName, 'w', zipfile.ZIP_DEFLATED)
#使用zipfile模块创建一个新的zip文件对象，指定为写模式('w')并采用ZIP_DEFLATED压缩算法。

# 遍历指定路径下的所有文件和文件夹
for dirpath, dirnames, filenames in os.walk(path): #使用os.walk函数遍历指定路径下的所有文件和文件夹，包括子文件夹
    for filename in filenames: #遍历每个文件夹中的文件名。
        print(filename)
        # 将文件添加到zip文件中
        f.write(os.path.join(dirpath, filename))

# 关闭zip文件
f.close()